In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
import os
os.chdir('/content/gdrive/My Drive/ieeebigmm/')

In [3]:
! pip install demoji

In [4]:
from loaders import *
from load_data.data_load import *
from load_data.load_preprocess import *
from load_data.imports import * 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# !pip install pytorch_lightning
# !pip install -U sentence-transformers

In [6]:
# !pip install contractions

In [7]:
# import collections
# from PIL import Image
# import contractions

In [8]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer
# from loaders import *
import yaml

In [9]:
path='/content/gdrive/My Drive/ieeebigmm/'

In [10]:
import demoji
demoji.download_codes()

... OK (Got response in 0.27 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [11]:
def preprocess(s):
  def lower_case(s):
    return s.lower()
  def remove_com_url(s):
    x=re.sub(r'[\S]+[\s]+[\S]+.com[\S]*','',s) # remove url and the word before it. 
    return x # if url in beginning of string.
  def remove_brackets(s):
    """
    removes brackets and anything inside them.
    """
    return re.sub(r'\([^()]*\)', '', s)
  def remove_punct(s):
    s=re.sub(r'[@?!.,;:-]',' ',s) 
    s=re.sub(r'[\s]+',' ',s)
    s=' '.join(s.split())
    s=s.replace('\u201c','') # italic quotation marks.
    s=s.replace('"','')
    s=s.replace('‘','\'')
    return s
  #expand contractions.
  def remove_num_and_whitespace(s):
    s=re.sub(r'[0-9]+',' ',s)
    s=re.sub(r'[\s]+',' ',s)
    return s.strip()
  def remove_emojis(s):
    s=demoji.replace(s,' ')
    s=re.sub(r'[\s]+',' ',s)
    return s.strip()
  s=lower_case(s)
  s=remove_com_url(s)
  s=remove_brackets(s)
  s=remove_punct(s)
  s=remove_num_and_whitespace(s)
  s=remove_emojis(s)
  return s  

In [12]:
# data loading and bactching
gossip_cop_test=pd.read_csv(path+'gossip_test_data_participant.csv',sep='\t')
politi_test=pd.read_csv(path+'politifact_test_data_participant.csv',sep='\t')

In [13]:
politi_null_title=np.where(politi_test['title'].isnull()==True)[0]
politi_test.loc[politi_null_title.tolist(),'title']=politi_test.loc[politi_null_title.tolist(),'text']

In [14]:
null_title=np.where(gossip_cop_test['title'].isnull()==True)[0]
gossip_cop_test.loc[null_title.tolist(),'title']=gossip_cop_test.loc[null_title.tolist(),'text']

In [15]:
gossip_cop_test['cleaned_title']=gossip_cop_test['title'].apply(preprocess)
gossip_cop_test['cleaned_text']=gossip_cop_test['text'].apply(preprocess)
politi_test['cleaned_title']=politi_test['title'].apply(preprocess)

In [16]:
def expand(x):
  l=[contractions.fix(word) for word in x.split()]
  return ' '.join(l)
# remove ’s.
gossip_cop_test['cleaned_title']=gossip_cop_test['cleaned_title'].apply(lambda x: re.sub(r'’s','\'s',x))
# gossip_cop_test['cleaned_text']=gossip_cop_test['cleaned_text'].apply(lambda x: re.sub(r'’s','\'s',x))
politi_test['cleaned_title']=politi_test['cleaned_title'].apply(lambda x: re.sub(r'’s','\'s',x))

# replace & with and 
gossip_cop_test['cleaned_title']=gossip_cop_test['cleaned_title'].apply(lambda x: re.sub(r'&','and',x))
politi_test['cleaned_title']=politi_test['cleaned_title'].apply(lambda x: re.sub(r'&','and',x))

# expand contractions.
gossip_cop_test['cleaned_title']=gossip_cop_test['cleaned_title'].apply(expand)
politi_test['cleaned_title']=politi_test['cleaned_title'].apply(expand)

In [17]:
stop_words=set(stopwords.words('english'))

In [18]:
def remove_stopwords(x):
  tokenised=word_tokenize(x)
  no_stopwords=[word for word in tokenised if word not in stop_words]
  return ' '.join(no_stopwords)
gossip_cop_test['cleaned_title']=gossip_cop_test['cleaned_title'].apply(remove_stopwords)
politi_test['cleaned_title']=politi_test['cleaned_title'].apply(remove_stopwords)

In [19]:
from sentence_transformers import SentenceTransformer,models
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
model_conv = torchvision.models.resnet18(pretrained=True)

100%|██████████| 405M/405M [00:06<00:00, 65.3MB/s]
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [20]:
# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Load pre-trained model 
bert_model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True, )
if torch.cuda.is_available():
  bert_model=bert_model.cuda()
bert_model.eval();  

### Infer

In [21]:
def seed_everything(seed=123):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
      torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [22]:
class baseline_pretrained():
  @staticmethod
  def img_feature(img):
    img=model_conv.conv1(img)
    img=model_conv.bn1(img)
    img=model_conv.layer1(img)
    img=model_conv.layer2(img)
    img=model_conv.layer3(img)
    img=model_conv.layer4(img)
    img=model_conv.avgpool(img)

    return img.view(img.shape[0],-1)
  @staticmethod
  def text_feature(text,is_cuda):
    feature=torch.tensor(np.array(sbert_model.encode(text)))
    if is_cuda:
      return feature.cuda()
    return feature 
class visual_attention_pretrained():
  @staticmethod
  def img_feature(img):
    img=model_conv.conv1(img)
    img=model_conv.bn1(img)
    img=model_conv.layer1(img)
    img=model_conv.layer2(img)
    img=model_conv.layer3(img)
    img=model_conv.layer4(img)
    img=img.view(img.shape[0],img.shape[1],-1)
    return img                        
  @staticmethod
  def text_feature(text,is_cuda):
    feature=torch.tensor(np.array(sbert_model.encode(text)))
    if is_cuda:
      return feature.cuda()
    return feature     

In [23]:
def bert_sentence_embedding(text, model):
  cuda=torch.device('cuda')
  twc = ["[CLS] " + txt + " [SEP]" for txt in text]

  # Tokenize the sentence with the BERT tokenizer
  tx = [tokenizer.tokenize(text_with_cls) for text_with_cls in twc]
  
  tt=[]
  for l in tx:
    if len(l)>512:
      tt.append(l[:300]+l[-200:])
    else:
      tt.append(l)  


  # Map the token strings to their vocabulary indeces
  it = [tokenizer.convert_tokens_to_ids(tokenized_text) for tokenized_text in tt]

  # for tup in zip(tokenized_text, indexed_tokens):
    # print('{:<12} {:>6,}'.format(tup[0], tup[1]))

  #token_type_ids to separate one sentence from other
  sids = [[1] * len(tokenized_text) for tokenized_text in tt]

  tokens_tensor = [torch.tensor([indexed_tokens]).to(cuda) for indexed_tokens in it]
  segments_tensors = [torch.tensor([segments_ids]).to(cuda) for segments_ids in sids]

  with torch.no_grad():

    outputs = [model(t_tensor, s_tensors) for t_tensor,s_tensors in zip(tokens_tensor,segments_tensors)]
    # contains outputs of all layers
    hidden_states = [output[2] for output in outputs]

    """
    Current dims:
    [# layers, # batches, # tokens, # features] 
    in our case batches = 1 as only one sentence in being passed
    """
    token_vecs = [hidden_state[-2][0] for hidden_state in hidden_states]

    # Calculate the average of all 22 token vectors.
    sentence_embedding = [(torch.mean(token_vec, dim=0)).unsqueeze(0) for token_vec in token_vecs]

    return torch.cat(sentence_embedding,dim=0)

In [24]:
class baseline_lightning(pl.LightningModule):
  def __init__(self,*args,**kwargs):
    
    super().__init__()

    self.hparams=kwargs
    img_dim=kwargs['img_dim']
    text_dim=kwargs['text_dim']

    self.feature_extract=baseline_pretrained()
    self.is_cuda=args[0]

    seed_everything(self.is_cuda)

    self.text_weight=nn.Parameter(torch.empty(text_dim,self.hparams.equalise_dim))
    torch.nn.init.kaiming_normal_(self.text_weight, mode='fan_out', nonlinearity='leaky_relu')
                                                          
    self.img_weight=nn.Parameter(torch.empty(img_dim,self.hparams.equalise_dim))
    torch.nn.init.kaiming_normal_(self.img_weight, mode='fan_out', nonlinearity='leaky_relu')

    self.fc_1=nn.Linear(2*self.hparams.equalise_dim,300)
    self.fc_2=nn.Linear(300,100)
    self.fc_3=nn.Linear(100,1)
    self.sig=nn.Sigmoid()
    self.loader_len=args[1]
    self.drop_last=True

  def forward(self,img,text,mode='pre_hoc'):
    img_feature=self.feature_extract.img_feature(img)

    if self.hparams.text_model=='sbert':
      title_feature=self.feature_extract.text_feature(text,self.is_cuda)
    if self.hparams.text_model=='bert':
      title_feature=bert_sentence_embedding(text, bert_model)
    
    if mode=='post_hoc':
      title_feature.requires_grad=True

    img=torch.mm(img_feature,self.img_weight)
    title=torch.mm(title_feature,self.text_weight)

    combined=torch.cat((img,title),1)

    out=F.relu(self.fc_1(combined))
    out=F.relu(self.fc_2(out))
    out=self.fc_3(out)
    if mode=='post_hoc':
      return out,title_feature,img_feature
    return out
  def bce_loss(self,logits,labels):
    pos_weight=torch.tensor([self.hparams.pos_weight])
    if self.is_cuda:
      pos_weight=pos_weight.cuda()
    return F.binary_cross_entropy_with_logits(logits,labels,pos_weight=pos_weight)

  def training_step(self, train_batch, batch_idx):

    img,text,label=train_batch
    logits=self.forward(img,text)

    loss=self.bce_loss(logits.squeeze(1),label)

    pred=self.logits_to_class(logits)

    # f1=self.metric(pred,label)
    logs = {'train_loss': loss}
    return {'loss': loss, 'log': logs}

  def validation_step(self,val_batch,val_idx): 

    img,text,label=val_batch
    logits=self.forward(img,text)
    loss=self.bce_loss(logits.squeeze(1),label)

    pred=self.logits_to_class(logits)
    # f1=self.metric(pred,label)

    return {'val_loss': loss}

  def validation_epoch_end(self, outputs):

    # called at the end of the validation epoch
    # outputs is an array with what you returned in validation_step for each batch
    # outputs = [{'loss': batch_0_loss}, {'loss': batch_1_loss}, ..., {'loss': batch_n_loss}] 
    avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    # avg_f1=torch.stack([x['val_f1'] for x in outputs]).mean()

    tensorboard_logs = {'val_loss': avg_loss}
    return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}   

  def test_step(self,test_batch,test_idx):
    img,text,label=val_batch
    logits=self.forward(img,text)
    loss=self.bce_loss(logits.squeeze(1),label)

    pred=self.logits_to_class(logits)
    # f1=self.metric(pred,label)
    return {'test_loss': loss}

  def test_epoch_end(self,outputs):
    avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
    # avg_f1 = torch.stack([x['test_f1'] for x in outputs]).mean()
    tensorboard_logs = {'test_loss': avg_loss}
    return {'avg_test_loss': avg_loss, 'log': tensorboard_logs}  

  def configure_optimizers(self):
    optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate,weight_decay=self.hparams.weight_decay)
    scheduler=torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=self.loader_len, epochs=self.hparams.epochs)
    return [optimizer],[scheduler]   

  def prepare_data(self):  
    path='/content/gdrive/My Drive/ieeebigmm/'
    
    transform = transforms.Compose([transforms.Resize(255),                       
                              transforms.CenterCrop(224),                   
                              transforms.ToTensor()]) 
    
    seed_everything(self.is_cuda)
    self.train_dataset=create_dataset(path,transform,X_train,self.hparams.col_name,'images',y_train)
    self.val_dataset=create_dataset(path,transform,X_val,self.hparams.col_name,'images',y_val)
    self.test_dataset=create_dataset(path,transform,X_test,self.hparams.col_name,'images',y_test)

  def train_dataloader(self):
    return DataLoader(dataset=self.train_dataset, shuffle=True, batch_size=self.hparams.batch_size,drop_last=self.drop_last,num_workers=4,pin_memory=True)
  
  def val_dataloader(self):
    return DataLoader(dataset=self.val_dataset, shuffle=False, batch_size=self.hparams.batch_size,drop_last=self.drop_last,num_workers=4,pin_memory=True)
  def test_dataloader(self):
    return DataLoader(dataset=self.test_dataset, shuffle=False, batch_size=self.hparams.batch_size,drop_last=self.drop_last)  
  def logits_to_class(self,logits):
    logits=self.sig(logits)
    logits[logits>=0.5]=1.
    logits[logits<0.5]=0.  
    return logits
  def score(self,prediction,actual):
      prediction=self.sig(prediction)
      pred=prediction.squeeze(1).cpu().detach().numpy()
      pred[pred>=0.5]=1
      pred[pred<0.5]=0                                                          
      targ=actual.cpu().numpy()
      f1=f1_score(targ,pred) 
      precision=precision_score(targ,pred)
      return f1,precision

In [27]:
class visual_attention_lightning(pl.LightningModule):
  def __init__(self,*args,**kwargs):
    self.hparams=kwargs

    img_dim=kwargs['img_dim']
    text_dim=kwargs['text_dim']


    super().__init__()

    self.feature_extract=visual_attention_pretrained()
    self.is_cuda=args[0]

    seed_everything(self.is_cuda)

    self.equalise_text=nn.Linear(text_dim,self.hparams.equalise_dim)
    self.equalise_img=nn.Linear(img_dim,self.hparams.equalise_dim)

    self.affinity_weight=nn.Parameter(torch.randn(self.hparams.equalise_dim,self.hparams.equalise_dim))
    nn.init.xavier_normal_(self.affinity_weight)

    k=self.hparams.k
    self.visual_weight=nn.Parameter(torch.randn(k,self.hparams.equalise_dim))
    nn.init.xavier_normal_(self.visual_weight)

    self.text_weight=nn.Parameter(torch.randn(k,self.hparams.equalise_dim))
    nn.init.xavier_normal_(self.text_weight)

    self.attention_weight=nn.Parameter(torch.randn(1,k))
    nn.init.xavier_normal_(self.attention_weight)

    self.softmax=nn.Softmax(dim=2)

    self.sig=nn.Sigmoid()

    self.fc_1=nn.Linear(self.hparams.equalise_dim,50)
    self.fc_2=nn.Linear(50,1)

    self.drop_last=True

    self.loader_len=args[1]
  def forward(self,img,text,mode='pre_hoc'):
    img_f=self.feature_extract.img_feature(img)
    if self.hparams.text_model=='sbert':
      text_f=self.feature_extract.text_feature(text,self.is_cuda)
    if self.hparams.text_model=='bert':
      text_f=bert_sentence_embedding(text,bert_model)  

    img_equalised=(self.equalise_img(img_f.transpose(1,2))).transpose(1,2)

    if mode=='post_hoc':
      text_f.requires_grad=True

    text_equalised=self.equalise_text(text_f).unsqueeze(-1)
    
    affinity_matrix=(torch.tanh(torch.bmm(text_equalised.transpose(1,2),torch.bmm(self.affinity_weight.repeat(img.shape[0],1,1),img_equalised))))

    h=torch.tanh(torch.bmm(self.visual_weight.repeat(img.shape[0],1,1),img_equalised) + torch.bmm(torch.bmm(self.text_weight.repeat(img.shape[0],1,1),text_equalised),affinity_matrix))

    visual_attention_scores=self.softmax(torch.bmm(self.attention_weight.repeat(img.shape[0],1,1),h))

    weighted_sum_image_feature=(img_equalised*visual_attention_scores).sum(dim=2)
    
    out=F.relu(self.fc_1(weighted_sum_image_feature))
    out=self.fc_2(out)

    return out 

  def bce_loss(self,logits,labels):
    pos_weight=torch.tensor([self.hparams.pos_weight])
    if self.is_cuda:
      pos_weight=pos_weight.cuda()
    return F.binary_cross_entropy_with_logits(logits,labels,pos_weight=pos_weight)

  def training_step(self, train_batch, batch_idx):

    img,text,label=train_batch
    logits=self.forward(img,text)

    loss=self.bce_loss(logits.squeeze(1),label)

    pred=self.logits_to_class(logits)

    # f1=self.metric(pred,label)
    logs = {'train_loss': loss}
    return {'loss': loss, 'log': logs}

  def validation_step(self,val_batch,val_idx): 

    img,text,label=val_batch
    logits=self.forward(img,text)
    loss=self.bce_loss(logits.squeeze(1),label)

    pred=self.logits_to_class(logits)
    # f1=self.metric(pred,label)

    return {'val_loss': loss}
    

  def validation_epoch_end(self, outputs):

    # called at the end of the validation epoch
    # outputs is an array with what you returned in validation_step for each batch
    # outputs = [{'loss': batch_0_loss}, {'loss': batch_1_loss}, ..., {'loss': batch_n_loss}] 
    avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    # avg_f1=torch.stack([x['val_f1'] for x in outputs]).mean()

    tensorboard_logs = {'val_loss': avg_loss}
    return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}   

  def test_step(self,test_batch,test_idx):
    img,text,label=val_batch
    logits=self.forward(img,text)
    loss=self.bce_loss(logits.squeeze(1),label)

    pred=self.logits_to_class(logits)
    # f1=self.metric(pred,label)
    return {'test_loss': loss}

  def test_epoch_end(self,outputs):
    avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
    # avg_f1 = torch.stack([x['test_f1'] for x in outputs]).mean()
    tensorboard_logs = {'test_loss': avg_loss}
    return {'avg_test_loss': avg_loss, 'log': tensorboard_logs}  

  def configure_optimizers(self):
    optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate,weight_decay=self.hparams.weight_decay)
    scheduler=torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=self.loader_len, epochs=self.hparams.epochs)
    return [optimizer],[scheduler]   

  def prepare_data(self):  
    path='/content/gdrive/My Drive/ieeebigmm/'
    
    transform = transforms.Compose([transforms.Resize(255),                       
                              transforms.CenterCrop(224),                   
                              transforms.ToTensor()]) 
    
    seed_everything(self.is_cuda)
    self.train_dataset=create_dataset(path,transform,X_train,self.hparams.col_name,'images',y_train)
    self.val_dataset=create_dataset(path,transform,X_val,self.hparams.col_name,'images',y_val)
    self.test_dataset=create_dataset(path,transform,X_test,self.hparams.col_name,'images',y_test)

  def train_dataloader(self):
    return DataLoader(dataset=self.train_dataset, shuffle=True, batch_size=self.hparams.batch_size,drop_last=self.drop_last,num_workers=3,pin_memory=True)
  
  def val_dataloader(self):
    return DataLoader(dataset=self.val_dataset, shuffle=False, batch_size=self.hparams.batch_size,drop_last=self.drop_last,num_workers=3,pin_memory=True)
  def test_dataloader(self):
    return DataLoader(dataset=self.test_dataset, shuffle=False, batch_size=self.hparams.batch_size,drop_last=self.drop_last)  
  def logits_to_class(self,logits):
    logits=self.sig(logits)
    logits[logits>=0.5]=1.
    logits[logits<0.5]=0.  
    return logits
  def score(self,prediction,actual):
      prediction=self.sig(prediction)
      pred=prediction.squeeze(1).cpu().detach().numpy()
      pred[pred>=0.5]=1
      pred[pred<0.5]=0                                                          
      targ=actual.cpu().numpy()
      f1=f1_score(targ,pred) 
      precision=precision_score(targ,pred)
      return f1,precision    

In [28]:
is_cuda=torch.cuda.is_available()
loader_len=100 # does not matter for inference.
args=[is_cuda,loader_len]

In [29]:
# version 26 is visual attention 0.87 f1 and 0.84 prec.
# version 33 is baseline 0.89 f1 and 0.85 prec.
# version 50 is viz_attn pos_weight=0.5. f1=0.87 prec=0.86
# version 51 is baseline with pos_weight=0.5 f1=0.84 prec=0.89
# version 53 is baseline with pos_weight=1. f1=0.92 prec=0.92 # politi. 33 finetuned

def load_model_inference(version,path,is_cuda,args,model_type=None):
  checkpoint_path=path+'/lightning_logs/version_'+str(version)+'/checkpoints/'+ os.listdir(path+'/lightning_logs/version_'+str(version)+'/checkpoints')[0]

  with open(path+'/lightning_logs/version_'+str(version)+'/hparams.yaml') as file:
    param_dict=yaml.load(file, Loader=yaml.FullLoader)

  assert(model_type is not None),'Enter the type of model you want to infer'

  if model_type=='baseline':
    test_model=baseline_lightning(*args,**param_dict)
  if model_type=='viz_attn':
    test_model=visual_attention_lightning(*args,**param_dict)  
  checkpoint = torch.load(checkpoint_path, map_location=lambda storage, loc: storage)

  test_model.load_state_dict(checkpoint['state_dict'])

  if is_cuda:
    test_model=test_model.cuda()
   
  test_model.eval()   
  return test_model

In [30]:
politi_53=load_model_inference(53,path,is_cuda,args,'baseline')

In [ ]:
# baseline_test=load_model_inference(33,path,is_cuda,args,'baseline')
# vis_attn_test=load_model_inference(50,path,is_cuda,args,'viz_attn')
# baseline_politifact=load_model_inference(53,path,is_cuda,args,'baseline')
# baseline_test_pos=load_model_inference(51,path,is_cuda,args,'baseline')
# vis_attn_26=load_model_inference(26,path,is_cuda,args,'viz_attn')
baseline_33=load_model_inference(33,path,is_cuda,args,'baseline')
baseline_61=load_model_inference(61,path,is_cuda,args,'baseline')

baseline_67=load_model_inference(67,path,is_cuda,args,'baseline')
baseline_68=load_model_inference(68,path,is_cuda,args,'baseline')

In [ ]:
vis_atnn_69=load_model_inference(69,path,is_cuda,args,'viz_attn')
vis_atnn_72=load_model_inference(72,path,is_cuda,args,'viz_attn')

vis_atnn_73=load_model_inference(73,path,is_cuda,args,'viz_attn')
vis_atnn_74=load_model_inference(74,path,is_cuda,args,'viz_attn')

In [33]:
baseline_33=load_model_inference(33,path,is_cuda,args,'baseline')

In [ ]:
baseline_75=load_model_inference(75,path,is_cuda,args,'baseline') # new 61.

In [ ]:
vis_attn_78=load_model_inference(78,path,is_cuda,args,'viz_attn')

In [ ]:
vis_atnn_73.hparams

"batch_size":    32
"col_name":      cleaned_text
"epochs":        10
"equalise_dim":  100
"img_dim":       512
"k":             10
"learning_rate": 0.0001
"pos_weight":    1.0
"text_dim":      768
"text_model":    sbert
"weight_decay":  0.0001

In [31]:
def num_params(model):
  total=0
  for param in model.parameters():
    total+=param.numel()
  return total  

In [32]:
num_params(politi_53)

406501

In [ ]:
num_params(vis_atnn_74)

145311

In [ ]:
baseline_76=load_model_inference(76,path,is_cuda,args,'baseline') # new 67
baseline_77=load_model_inference(77,path,is_cuda,args,'baseline') # new 68

In [ ]:
baseline_76.hparams

"batch_size":    32
"col_name":      cleaned_text
"epochs":        10
"equalise_dim":  200
"img_dim":       512
"learning_rate": 0.0006
"pos_weight":    1.0
"text_dim":      768
"text_model":    bert
"weight_decay":  0.0001

In [ ]:
baseline_77.hparams

"batch_size":    32
"col_name":      cleaned_text
"epochs":        10
"equalise_dim":  200
"img_dim":       512
"learning_rate": 0.0006
"pos_weight":    1.0
"text_dim":      768
"text_model":    sbert
"weight_decay":  0.0001

In [46]:
if is_cuda:
  model_conv=model_conv.cuda()
  sbert_model=sbert_model.cuda()
  bert_model=bert_model.cuda()

In [ ]:
def display_torch(img):
  img=img.squeeze(0).permute(1,2,0).detach().cpu().numpy()
  pyplot.imshow(img)

In [ ]:
# class create_dataset(datasets.ImageFolder):
#   def __init__(self,path,transform,x,column_name,img_folder,y=None,seed=123):

#     self.root=path
#     custom_samples=[]

#     downloaded=indx_to_img_name(x)

#     filenames=list(np.char.add(path+img_folder+'/',downloaded))  
#     self.test_time=y  
#     if y is not None:    
#       labels=list(y.values)

#     text=x[column_name].values
#     if y is not None:
#       for title,img_path,label in zip(text,filenames,labels):
#         custom_samples.append((img_path,title,torch.tensor(label,dtype=float)))  
#     else:
#       for title,img_path in zip(text,filenames):
#         custom_samples.append((img_path,title))     

#     self.samples=custom_samples
#     self.transform=transform
#   def __getitem__(self,idx):  
#     if self.test_time is not None:
#       img_path,text,label=self.samples[idx]
#     else:
#       img_path,text=self.samples[idx]

   
#     img= Image.open(img_path)
#     img=img.convert('RGB')
#     if self.transform is not None:
#         img=self.transform(img)
#     if self.test_time is not None:
#       return (img,text,label)    
#     else:
#       return (img,text)  
#   def __len__(self):
#     return len(self.samples)   

In [37]:
transform = transforms.Compose([transforms.Resize(255),                       
                                  transforms.CenterCrop(224),                   
                                  transforms.ToTensor()]) 
sig=nn.Sigmoid()

In [ ]:
def model_predictions(model,img_folder='gossip_test_data',batch_size=128):
  if torch.cuda.is_available():
    model=model.cuda()
  infer_dataset=create_dataset(path,transform,gossip_cop_test,model.hparams.col_name,img_folder)
  infer_loader=DataLoader(dataset=infer_dataset,batch_size=batch_size,drop_last=False,shuffle=False)
  predictions=[]
  for i,(img,text) in enumerate(infer_loader):
    print(i)
    if is_cuda:
      img=img.cuda()
    with torch.no_grad():
      pred_1=sig(model(img,text))  
      pred_1[pred_1>=0.5]=1
      pred_1[pred_1<0.5]=0
      predictions.append(pred_1)

  return torch.cat(predictions)

In [34]:
def submission_file(tensor_preds,filename,test_file='gossip_test_data_participant.csv'):
  path='/content/gdrive/My Drive/ieeebigmm/'
  submission=pd.read_csv(path+test_file,sep='\t')
  submission['label']=tensor_preds.cpu().numpy()
  final_submission=submission[['title','label']]
  final_submission.to_csv(path+filename+'.csv',index=False)

In [ ]:
model_predictions(baseline_6)

In [35]:
test_file='politifact_test_data_participant.csv'

In [50]:
submission_file(politi_preds,'politi_33',test_file=test_file)

In [ ]:
vis_atnn_74.hparams.col_name

'cleaned_text'

In [43]:
gossip_dataset=create_dataset(path,transform,gossip_cop_test,'cleaned_title','gossip_test_data')
politi_dataset=create_dataset(path,transform,politi_test,'cleaned_title','politi_test_data')

In [44]:
batch_size=128
gossip_loader= DataLoader(dataset=gossip_dataset,batch_size=batch_size,drop_last=False)
politi_loader= DataLoader(dataset=politi_dataset,batch_size=batch_size,drop_last=False)

In [47]:
sig=nn.Sigmoid()
predictions=[]
for i,(img,title) in enumerate(politi_loader):
  if is_cuda:
    img=img.cuda()
  with torch.no_grad():
    pred_1=sig(baseline_33(img,title))  
    pred_1[pred_1>=0.5]=1
    pred_1[pred_1<0.5]=0
    predictions.append(pred_1)

In [48]:
politi_preds=torch.cat(predictions)

In [49]:
politi_preds.shape 

torch.Size([138, 1])

In [ ]:
is_cuda

True

In [ ]:
gossip_cop_test.head()['cleaned_title']

In [ ]:
baseline_67.hparams

"batch_size":    32
"col_name":      cleaned_text
"epochs":        10
"equalise_dim":  200
"img_dim":       512
"learning_rate": 0.0001
"pos_weight":    1.0
"text_dim":      768
"text_model":    bert
"weight_decay":  0.0001

In [ ]:
gossip_predictions=[]
for i,(img,title) in enumerate(gossip_loader):
  print(i)
  if is_cuda:
    img=img.cuda()
  with torch.no_grad():
    pred_1=(vis_attn_78(img,title))  
    # pred_2=(baseline_test_pos(img,title))

    # final_pred=sig((pred_1+pred_2)/2)
    final_pred=sig(pred_1)

    final_pred[final_pred>=0.5]=1
    final_pred[final_pred<0.5]=0

    gossip_predictions.append(final_pred)

0
1
2
3


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


4
5
6
7


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


8
9
10


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 19. Skipping tag 42036
  " Skipping tag %s" % (size, len(data), tag)


11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [ ]:
gossip_preds=torch.cat(gossip_predictions)

In [ ]:
(gossip_predictions[0]==gossip_preds[:128]).sum()

tensor(128, device='cuda:0')

In [ ]:
gossip_preds.shape

torch.Size([3827, 1])

In [ ]:
for i in range (gossip_cop_test.shape[0]):
  try:
    x=gossip_dataset[i]
  except:
    print(i)  

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning:

Palette images with Transparency expressed in bytes should be converted to RGBA images

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning:

Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 19. Skipping tag 42036



3613


In [ ]:
gossip_dataset[3613]

(tensor([[[0.4706, 0.4706, 0.4784,  ..., 0.4745, 0.4745, 0.4745],
          [0.4706, 0.4706, 0.4784,  ..., 0.4745, 0.4745, 0.4745],
          [0.4863, 0.4863, 0.4902,  ..., 0.4824, 0.4824, 0.4824],
          ...,
          [0.4471, 0.4471, 0.4510,  ..., 0.4510, 0.4510, 0.4510],
          [0.4471, 0.4471, 0.4510,  ..., 0.4549, 0.4510, 0.4510],
          [0.4471, 0.4471, 0.4510,  ..., 0.4549, 0.4549, 0.4510]],
 
         [[0.4314, 0.4314, 0.4392,  ..., 0.4353, 0.4353, 0.4353],
          [0.4314, 0.4314, 0.4392,  ..., 0.4353, 0.4353, 0.4353],
          [0.4353, 0.4353, 0.4392,  ..., 0.4392, 0.4392, 0.4392],
          ...,
          [0.3922, 0.3922, 0.3961,  ..., 0.3961, 0.3961, 0.3961],
          [0.3922, 0.3922, 0.3961,  ..., 0.4000, 0.3961, 0.3961],
          [0.3922, 0.3922, 0.3961,  ..., 0.4000, 0.4000, 0.3961]],
 
         [[0.4235, 0.4235, 0.4314,  ..., 0.4314, 0.4314, 0.4314],
          [0.4235, 0.4235, 0.4314,  ..., 0.4314, 0.4314, 0.4314],
          [0.4314, 0.4314, 0.4275,  ...,

In [ ]:
# pol_submission=pd.read_csv(path+'politifact_test_data_participant.csv',sep='\t')

In [ ]:
# pol_submission['label']=politi_preds.cpu().numpy()

In [ ]:
# final_pol_submission=pol_submission[['title','label']]

In [ ]:
# final_pol_submission.to_csv(path+'politi_submission.csv',index=False)

In [ ]:
gossip_submission=pd.read_csv(path+'gossip_test_data_participant.csv',sep='\t')
gossip_submission['label']=gossip_preds.cpu().numpy()

In [ ]:
final_gossip_submission=gossip_submission[['title','label']]
final_gossip_submission

,title,label
0,Justin Timberlake Wants ‘as Many Kids’ as He C...,0.0
1,TIME's 30 Most Influential Teens of 2017,1.0
2,Nikki Bella Shares Romantic Photo with Ex John...,1.0
3,Flip or Flop,1.0
4,'Double Dare' is coming back to Nickelodeon!,1.0
...,...,...
3822,Blake Griffin Warned By L.A. Clippers Bosses ‘...,1.0
3823,Emmys 2018: ‘Schitt’s Creek’ Star Dan Levy on ...,1.0
3824,This Timeline Of Khloe Kardashian's Pregnancy ...,0.0
3825,Jenna Dewan,1.0


In [ ]:
def submit(filename):
  final_gossip_submission.to_csv(path+filename+'.csv',index=False)
submit('gossip_submission_26')  

In [ ]:
df=pd.read_csv(path+'gossip_submission.csv')

In [ ]:
  #   final_pred=pred_1
  #   # pred_2=vis_attn_test(img,title)
  #   # final_pred=(sig((pred_1+pred_2)/2)).item() # ensemble. 
  #   if final_pred>=0.5:
  #     final_pred=1.
  #   else:
  #     final_pred=0. 

In [ ]:
vis_atnn_73.hparams

"batch_size":    32
"col_name":      cleaned_text
"epochs":        10
"equalise_dim":  100
"img_dim":       512
"k":             10
"learning_rate": 0.0001
"pos_weight":    1.0
"text_dim":      768
"text_model":    sbert
"weight_decay":  0.0001

In [ ]:
vis_atnn_74.hparams

"batch_size":    32
"col_name":      cleaned_text
"epochs":        10
"equalise_dim":  100
"img_dim":       512
"k":             10
"learning_rate": 0.0001
"pos_weight":    1.0
"text_dim":      768
"text_model":    bert
"weight_decay":  0.0001

In [ ]:
baseline_33.hparams

"batch_size":    32
"col_name":      cleaned_title
"epochs":        7
"equalise_dim":  200
"img_dim":       512
"learning_rate": 0.0006
"pos_weight":    1.0
"text_dim":      768
"text_model":    sbert
"weight_decay":  0.0001

In [ ]:
baseline_61.hparams

"batch_size":    32
"col_name":      cleaned_title
"epochs":        7
"equalise_dim":  200
"img_dim":       512
"learning_rate": 0.0004
"pos_weight":    1.0
"text_dim":      768
"text_model":    bert
"weight_decay":  0.0001

In [ ]:
baseline_67.hparams

"batch_size":    32
"col_name":      cleaned_text
"epochs":        10
"equalise_dim":  200
"img_dim":       512
"learning_rate": 0.0001
"pos_weight":    1.0
"text_dim":      768
"text_model":    bert
"weight_decay":  0.0001

In [ ]:
baseline_68.hparams

"batch_size":    32
"col_name":      cleaned_text
"epochs":        10
"equalise_dim":  200
"img_dim":       512
"learning_rate": 0.0001
"pos_weight":    1.0
"text_dim":      768
"text_model":    sbert
"weight_decay":  0.0001

In [ ]:
# 75 is the new 61.
